<a href="https://colab.research.google.com/github/RickBarretto/llm-playground/blob/main/playgrounds/bode-7b" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Author: [Recogna NLP](https://huggingface.co/recogna-nlp)
- Source: [Collab](https://colab.research.google.com/drive/1uqVCED2wNPXIa7On0OAnghJNr13PUB5o?usp=sharing)
- Repository: [BODE](https://huggingface.co/recogna-nlp/bode-7b-alpaca-pt-br-no-peft)
- License: [MIT](https://huggingface.co/datasets/choosealicense/licenses/blob/main/markdown/mit.md)

In [ ]:
!pip install transformers
!pip install einops accelerate bitsandbytes
!pip install sentence_transformers
#!pip install git+https://github.com/huggingface/peft.git

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
#from peft import PeftModel, PeftConfig

llm_model = 'recogna-nlp/bode-7b-alpaca-pt-br-no-peft'
#llm_model = 'recogna-nlp/bode-7b-alpaca-pt-br'
hf_auth = '<HF_ACCESS_KEY>'
#config = PeftConfig.from_pretrained(llm_model)
model = AutoModelForCausalLM.from_pretrained(llm_model, trust_remote_code=True, return_dict=True, load_in_4bit=True, device_map='auto', token=hf_auth)
tokenizer = AutoTokenizer.from_pretrained(llm_model, token=hf_auth)
#model = PeftModel.from_pretrained(model, llm_model) # Caso ocorra o seguinte erro: "ValueError: We need an `offload_dir`... Você deve acrescentar o parâmetro: offload_folder="./offload_dir".


In [ ]:
model = model.eval()

In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Abaixo está uma instrução que descreve uma tarefa, juntamente com uma entrada que fornece mais contexto. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Entrada:
{input}

### Resposta:"""
    else:
        return f"""Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que complete adequadamente o pedido.

### Instrução:
{instruction}

### Resposta:"""

generation_config = GenerationConfig(
    temperature=0.5,
    top_p=0.75,
    num_beams=2,
    do_sample=True
)

def evaluate(instruction, input=None):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_length=300
    )
    for s in generation_output.sequences:
        output = tokenizer.decode(s)
        print("Resposta:", output.split("### Resposta:")[1].strip())

In [ ]:
evaluate("Responda com detalhes: O que é um bode?")
